In [26]:
import numpy as np

In [27]:
# no. of stimuli x2 for not-stimuli as well
nf = 2*2
stim = {
    0:"X",
    1:"Y",
    2:"NotX",
    3:"NotY"
}
# variables
dt = 0.1
a = 0.8

In [28]:
#generate a range of s and tau_star
def s_gen(tau_min=.1, tau_max=100., buff_max=None, k=20, ntau=50, g=0.0):
    c = (tau_max/tau_min)**(1./(ntau-1))-1
    tau_star = tau_min*(1+c)**np.arange(ntau).astype(float)
    s = 1/tau_star
    return [tau_star, s]
[tau_star, S] = s_gen()

In [29]:
def stim_gen(i):
    global nf
    stim_vec = np.zeros(nf)
    if i is not None:
        stim_vec[i] = 1
    return stim_vec

In [30]:
def input_update(a,s,s_idx,fi_IN: None):
    global M
    global F
    global P
    f_IN = stim_gen(fi_IN)
    #eq 1
    F[s_idx] += -s*F[s_idx] + f_IN
    F[s_idx] = np.where(F[s_idx]<0, 0, F[s_idx])
    #eq 2
    P_IN = np.dot(M[s_idx], f_IN)
    #eq 3
    P[s_idx] +=  s*P[s_idx] + P_IN# - P[-1]
    
    for past_stim in np.flatnonzero(F[s_idx]):
        #eq4 - make sure we're indexing M correctly
        M[s_idx][fi_IN, past_stim] = a * M[s_idx][fi_IN, past_stim] + (1-a) * F[s_idx][past_stim]
def detect_notstim(a,t,s,s_idx,fi_IN:None):
    global M
    global F
    global P
    #find stimulis from the past that are still active
    predicted = np.where(P[s_idx]>=1)[0]
    #iterate through them
    for p in predicted:
        if p == fi_IN:
            print(stim[p], "Observed & Predicted! at at s=%s and t=%s" %(s,t))
            P[s_idx][p] = 0
        else:
            P[s_idx][p] = 0
            if p < 2:
                print(stim[p+2], "predicted at s=%s and t=%s" %(s,t))
                #add 2 to get the not-stimuli equivalent
                input_update(a,s,s_idx,fi_IN=p+2)
            else:
                print(stim[p], "predicted at s=%s and t=%s" %(s,t))
                input_update(a,s,s_idx, fi_IN=p)
def timestep_update(a,t,S,fi_IN: None):
    for s_idx, s in enumerate(S):
        input_update(a,s,s_idx,fi_IN)
        detect_notstim(a,t,s,s_idx,fi_IN) 

In [31]:
M = np.zeros([len(S), nf, nf])
F = np.zeros([len(S), nf])
P = np.zeros([len(S), nf])
#alter this for different stimuli presentation at different times
f = [[0,1],[1,3],[0,7],[0,10.1]]

In [32]:
T = np.arange(0,10, dt)
fi_prev = None
for t in T:
    if f[0][1] == t:
        fi_IN = f[0][0]
        timestep_update(a,t,S,fi_IN=fi_IN)
        f.pop(0)
    else:
        timestep_update(a,t,S,fi_IN=None)

NotX predicted at s=0.7906043210907688 and t=4.2
NotX predicted at s=0.9102981779915207 and t=4.3
NotX predicted at s=0.6866488450042991 and t=4.3
NotX predicted at s=0.5963623316594633 and t=4.3
NotX predicted at s=0.5179474679231202 and t=4.4
NotX predicted at s=0.44984326689694387 and t=4.4
NotX predicted at s=0.390693993705461 and t=4.5
NotX predicted at s=0.3393221771895322 and t=4.5
NotX predicted at s=0.29470517025518045 and t=4.6000000000000005
NotX predicted at s=0.25595479226995305 and t=4.6000000000000005
NotX predicted at s=0.222299648252619 and t=4.6000000000000005
NotX predicted at s=0.19306977288832458 and t=4.6000000000000005
NotX predicted at s=0.08286427728546822 and t=4.6000000000000005
NotX predicted at s=0.071968567300115 and t=4.6000000000000005
NotX predicted at s=0.06250551925273956 and t=4.6000000000000005
NotX predicted at s=0.05428675439323845 and t=4.6000000000000005
NotX predicted at s=0.04714866363457379 and t=4.6000000000000005
NotX predicted at s=0.04094